## Data preprocessing

This notebook contains all code related to preprocessing of data, so it can be used later for Exploratory Data Analysis and Predictive Model Training

### Importing necessary libraries

In [23]:
import pandas as pd

### Loading data 

In [8]:
data_url = "https://github.com/Naio/aasa-stability-prediction/blob/master/data/raw/aasa_1stn_4lys_1bpi.xlsx?raw=true"

p1STN = pd.read_excel(data_url,sheet_name='1STN')
p4LYZ = pd.read_excel(data_url,sheet_name='4LYZ')
p1BPI = pd.read_excel(data_url,sheet_name='1BPI')

### Conveniently renaming stability and protein name columns

In [9]:
def rename_columns(dataframe):
    return dataframe.rename(columns={'Unnamed: 0': 'id', 'Y': 'stability'})

In [10]:
p1STN = rename_columns(p1STN)
p4LYZ = rename_columns(p4LYZ)
p1BPI = rename_columns(p1BPI)

### Checking for dataset metadata

For each dataset, there are 481 numeric columns, 10 for each of the 48 **_descriptors_**, and 1 for the **_stability change_**. One row is the data for the **_wild-type_** protein, while the remaining rows are the data of the **_mutations_**. There's an additional column that contains the identifier for each row. The identifier is either **_WT_**, which stands for "wild-type", or a descriptive name for the mutation. The number of examples per dataset is:
 - 1STN: 42 examples
 - 4LYZ: 51 examples 
 - 1BPI: 53 examples

In [34]:
p1STN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Columns: 482 entries, Unnamed: 0 to 10AASAf
dtypes: float64(481), object(1)
memory usage: 158.3+ KB


In [36]:
p4LYZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Columns: 482 entries, Unnamed: 0 to 10AASAf
dtypes: float64(481), object(1)
memory usage: 192.2+ KB


In [221]:
p1BPI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 50 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         53 non-null     object 
 1   stability  53 non-null     float64
 2   AASAK0     53 non-null     float64
 3   AASAHt     53 non-null     float64
 4   AASAHP     53 non-null     float64
 5   AASAP      53 non-null     float64
 6   AASApHi    53 non-null     float64
 7   AASApK'    53 non-null     float64
 8   AASAMw     53 non-null     float64
 9   AASAP1     53 non-null     float64
 10  AASARf     53 non-null     float64
 11  AASAm      53 non-null     float64
 12  AASAHnc    53 non-null     float64
 13  AASAEsm    53 non-null     float64
 14  AASAE1     53 non-null     float64
 15  AASAEt     53 non-null     float64
 16  AASAPa     53 non-null     float64
 17  AASAPb     53 non-null     float64
 18  AASAPt     53 non-null     float64
 19  AASAPC     53 non-null     float64
 20  AASACa     5

### Checking for missing values

There's no missing values in any dataset.

In [11]:
def number_of_missing_values(dataframe):
    return (
    dataframe.isnull() #Checks dataframe cell if it has a missing value. Returns a dataframe filled with boolean values. 
    .sum() # Returns the number of missing values per column.
    .sum() # Outputs the number of missing values of the whole dataframe.
    )

In [12]:
print("Number of missing values per dataset:")
print("  p1STN:", number_of_missing_values(p1STN))
print("  p4LYZ:", number_of_missing_values(p4LYZ))
print("  p1BPI:", number_of_missing_values(p1BPI))

Number of missing values per dataset:
  p1STN: 0
  p4LYZ: 0
  p1BPI: 0


### Data reduction

In order to inspect data in more detail, we first need to find a way to reduce the 480 columns. When collecting data, each descriptor of a protein is measured 10 times. So, we calculate the mean of the 10 measurements for each of the descriptors. Thus, our data of 480 columns is reduced to only 48. The 10 measurements for a descriptor are registered in ordered and contiguous columns of the dataset. An example is the first descriptor *_AASAK0_*, which measurements are *_1AASAK0_*, *_2AASAK0_*, ..., *_10AASAK0_*. All 3 datasets share the same format. The following functions are used to average the descriptors measurements.

In [13]:
#Returns the new dataframe containing the measurement means of the descriptors.
def average_measurements(df):
    #Creates a new dataframe from original 'name' and 'stability' columns
    new_df = df.loc[:, ['id', 'stability']]
    
    #Creates a dictionary that maps the descriptor name to its measurements mean.
    means = {descriptor_name: measurements_mean for descriptor_name, measurements_mean in descriptors_means(df)}
    
    #Creates columns for each descriptor in the new dataframe.
    return new_df.assign(**means)

In [14]:
#For each one of the 48 descriptors, this generator calculates the mean of its 10 measurements.
def descriptors_means(df):
    for i in range(48): #For each one of the 48 descriptors
        
        # Extracts a dataframe containing measurements for a single descriptor
        descriptor_measurements = df.iloc[:, 10 * i + 2: 10 * i + 12]
        # Gets descriptor name from the first column of the measurements dataframe
        descriptor_name = descriptor_measurements.columns[1][1:]
        
        # Calculates and yields the mean over the 10 measurement columns
        yield descriptor_name, descriptor_measurements.mean(axis ='columns') 
    

The following code reduces the data to 50 columns, including stability and protein name.

In [15]:
p1STN = average_measurements(p1STN)
p4LYZ = average_measurements(p4LYZ)
p1BPI = average_measurements(p1BPI)

### Exporting to preprocessed data to CSV

It is convenient to export the data already preprocessed so it can be used later for EDA and Predictive Modeling 

In [2]:
processed_data_path = "../../data/processed/"

In [24]:
p1STN.to_csv(path_or_buf = processed_data_path + 'p1STN' + '.csv', index=False)
p4LYZ.to_csv(path_or_buf = processed_data_path + 'p4LYZ' + '.csv', index=False)
p1BPI.to_csv(path_or_buf = processed_data_path + 'p1BPI' + '.csv', index=False)